In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cp
import lseg.data as ld
from lseg.data.discovery import Chain
from lseg.data.content import esg
import seaborn as sns


# DATA DOWNLOAD

In [19]:
# ========== CONFIG ==========
INDEX_CHAIN = "0#.SPX"   # S&P 500 chain
INDEX_RIC   = ".SPX"     # benchmark index
START_DATE  = "2015-06-01"
END_DATE    = "2025-06-01"     
EMISSIONS_COL_NAME = "Emissions Score"  
# ============================

ld.open_session()



<lseg.data.session.Definition object at 0x181943f0590 {name='workspace'}>

In [20]:
# ---- STEP 1: Constituents ----
cons_file = "constituents.csv"
if os.path.exists(cons_file):
    tickers = pd.read_csv(cons_file)["RIC"].tolist()
else:
    cons = Chain(INDEX_CHAIN)
    tickers = cons.constituents
    pd.DataFrame({"RIC": tickers}).to_csv(cons_file, index=False)

print("Number of constituents:", len(tickers))
print("Sample tickers:", tickers[:10])



Number of constituents: 503
Sample tickers: ['A.N', 'AAPL.OQ', 'ABBV.N', 'ABNB.OQ', 'ABT.N', 'ACGL.OQ', 'ACN.N', 'ADBE.OQ', 'ADI.OQ', 'ADM.N']


In [21]:

carbon_file = "spx500_carbon_emissions_snapshots.csv"

if os.path.exists(carbon_file):
    print(f"Loading cached carbon emissions data from {carbon_file}...")
    carbon_df = pd.read_csv(carbon_file)
else:
    print("Fetching ESG snapshot data from LSEG...")
    carbon_df = ld.get_data(universe=tickers, fields=["TR.CO2EmissionTotal "])
    carbon_df.to_csv(carbon_file, index=False)
    print(f"Saved carbon emissions data to {carbon_file}")

print(carbon_df.head())


Loading cached carbon emissions data from spx500_carbon_emissions_snapshots.csv...
  Instrument  CO2 Equivalent Emissions Total
0        A.N                         68619.0
1    AAPL.OQ                       1279700.0
2     ABBV.N                        541214.0
3    ABNB.OQ                          7363.0
4      ABT.N                        981000.0


In [29]:
# ===== FULL S&P500 PRICES (wide, batched, robust) =====
# Assumes: ld.open_session() already called, START_DATE, END_DATE defined.
# Uses existing `tickers` (list). If missing, loads from constituents.csv.

import pandas as pd
import numpy as np
import os

prices_file = "prices.csv"
BATCH_SIZE = 20
MAX_RETRIES = 2

if "tickers" not in globals() or not tickers:
    tickers = pd.read_csv("constituents.csv")["RIC"].dropna().tolist()

def fetch_one(univ):
    df = ld.get_history(
        universe=univ,
        fields=["TR.PriceClose"],
        start=START_DATE,
        end=END_DATE,
        interval="daily"
    )
    df.index = pd.to_datetime(df.index, errors="coerce")
    df.index.name = "Date"
    return df

all_blocks = []
failed_total = []

for i in range(0, len(tickers), BATCH_SIZE):
    batch = tickers[i:i+BATCH_SIZE]
    print(f"[{i//BATCH_SIZE+1}] batch of {len(batch)}")
    try:
        df = fetch_one(batch)
        all_blocks.append(df)
    except Exception as e:
        print(f"  batch failed → per‑ticker: {e}")
        for t in batch:
            ok = False
            retries = MAX_RETRIES
            while retries > 0 and not ok:
                try:
                    df1 = fetch_one([t])
                    all_blocks.append(df1)
                    ok = True
                except Exception as e1:
                    retries -= 1
                    if retries == 0:
                        print(f"   give up {t}: {e1}")
                        failed_total.append(t)

if not all_blocks:
    raise RuntimeError("No price data retrieved.")

# concat all wide blocks (align on Date)
prices_wide = pd.concat(all_blocks, axis=1)

# drop duplicated columns (keep first occurrence)
if prices_wide.columns.duplicated().any():
    prices_wide = prices_wide.loc[:, ~prices_wide.columns.duplicated()]

# optional cleanups
prices_wide = prices_wide.sort_index().dropna(axis=1, how="all")

prices_wide.to_csv(prices_file)
print(f"Saved {prices_file} | shape={prices_wide.shape} | failed={len(failed_total)}")
if failed_total:
    with open("prices_failed_ric.txt", "w") as f:
        f.write("\n".join(failed_total))
    print("Wrote prices_failed_ric.txt")

returns = prices_wide.pct_change().dropna()



[1] batch of 20


C:\Users\MasseiM\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\lseg\data\_tools\_dataframe.py:177:FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


[2] batch of 20


C:\Users\MasseiM\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\lseg\data\_tools\_dataframe.py:177:FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


[3] batch of 20


...

C:\Users\MasseiM\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\lseg\data\_tools\_dataframe.py:177:FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
C:\Users\MasseiM\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\lseg\data\_tools\_dataframe.py:192:FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


[24] batch of 20


C:\Users\MasseiM\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\lseg\data\_tools\_dataframe.py:177:FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


[25] batch of 20
  batch failed → per‑ticker: UDF Core request failed. Gateway Time-out Requested universes: ['WBD.OQ', 'WDAY.OQ', 'WDC.OQ', 'WEC.N', 'WELL.N', 'WFC.N', 'WM.N', 'WMB.N', 'WMT.N', 'WRB.N', 'WSM.N', 'WST.N', 'WTW.OQ', 'WY.N', 'WYNN.OQ', 'XEL.OQ', 'XOM.N', 'XYL.N', 'XYZ.N', 'YUM.N']. Requested fields: ['TR.PRICECLOSE']
[26] batch of 3
Saved prices.csv | shape=(2516, 464) | failed=0


C:\Users\MasseiM\AppData\Local\Temp\ipykernel_5780\3580421199.py:75:FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.


In [30]:
def fetch_fundamentals_in_batches(tickers, fields, out_file, batch_size=20):
    if os.path.exists(out_file):
        print(f"Loading cached data from {out_file}")
        return pd.read_csv(out_file)
    
    all_data = []
    for i in range(0, len(tickers), batch_size):
        batch = tickers[i:i + batch_size]
        print(f"Fetching batch {i//batch_size + 1}...")
        try:
            data = ld.get_data(
                universe=batch,
                fields=fields
            )
            all_data.append(data)
        except Exception as e:
            print(f"Batch {i//batch_size + 1} failed: {e}")
    df = pd.concat(all_data)
    df.to_csv(out_file, index=False)
    return df

# === CONFIG ===
FUND_FIELDS = ["TR.Revenue", "TR.EBIT", "TR.TRESGScore", "TR.EnvironmentPillarScore", "TR.SocialPillarScore", "TR.GovernancePillarScore"]
FUND_FILE = "fundamentals_esg_snapshot.csv"
tickers = carbon_df["Instrument"].dropna().unique().tolist()

# === FETCH ===
fund_df = fetch_fundamentals_in_batches(tickers, FUND_FIELDS, FUND_FILE)
print(fund_df.head())


Fetching batch 1...


C:\Users\MasseiM\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\lseg\data\_tools\_dataframe.py:192:FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


Fetching batch 2...


C:\Users\MasseiM\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\lseg\data\_tools\_dataframe.py:192:FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


Fetching batch 3...


...

Fetching batch 24...


C:\Users\MasseiM\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\lseg\data\_tools\_dataframe.py:192:FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


Fetching batch 25...


C:\Users\MasseiM\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\lseg\data\_tools\_dataframe.py:192:FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


Fetching batch 26...
  Instrument       Revenue            EBIT  ESG Score  \
0        A.N    6510000000    1608000000.0  80.006285   
1    AAPL.OQ  391035000000  123216000000.0  71.210659   
2     ABBV.N   56334000000   17407000000.0   71.81477   
3    ABNB.OQ   11102000000    2546000000.0  34.406651   
4      ABT.N   41950000000    7039000000.0  83.560889   

   Environmental Pillar Score  Social Pillar Score  Governance Pillar Score  
0                   77.626054            93.392301                63.492138  
1                   65.652592            65.005872                86.907587  
2                   64.122957            93.574354                44.936944  
3                   19.950904            60.430296                16.334124  
4                    78.94812            85.629443                82.989919  
